In [6]:
# Import standard libraries for data manipulation
import pandas as pd  # For data handling and analysis
import sys          # For system-specific parameters and functions
import os           # For operating system dependent functionality
import numpy as np  # For numerical operations and array handling
import matplotlib.pyplot as plt  # For plotting and image creation
from matplotlib.path import Path  # For polygon operations
from matplotlib.patches import PathPatch  # For creating filled polygons

# Add current directory to system path to allow importing custom modules
sys.path.append(os.path.abspath('.'))

import xml.etree.ElementTree as ET


<module 'tachytools' from 'd:\\Mestrado\\Dissertacao_Mestrado\\mestrado-rust\\Hulls\\tachytools.py'>

In [11]:
def load_xml(xml_file):
    """
    Load coordinates from XML file with option to include Z data.
    
    Parameters:
    xml_file: Path to the XML file
    rotate: Rotation angle in degrees (default: 0)
    mirror: Whether to mirror points over y-axis (default: False)
    remove_tachy_point: Whether to remove the last point (default: True)
    drop_points: Number of points to drop from the beginning (default: False)
    include_z: Whether to include Z coordinates in the return values (default: False)
    
    Returns:
    If include_z is False: x_data, y_data, timestamp (original behavior)
    If include_z is True: x_data, y_data, z_data, timestamp
    """
    # XML-Datei parsen
    tree = ET.parse(xml_file)
    root = tree.getroot()
    x_data = []
    y_data = []

    # Namespace-Definition für das XML-Parsing
    ns = {'landxml': 'http://www.landxml.org/schema/LandXML-1.2'}

    for cgpoint in root.findall('.//landxml:CgPoint', ns):
        coords = cgpoint.text.split()
        x_data.append(float(coords[0]))  # x-Koordinate
        y_data.append(float(coords[1]))  # y-Koordinate
    
    df = pd.DataFrame({'x_data': x_data, 'y_data': y_data})

    return df

In [7]:
def save_hull_image(hull_df, physical_cell_size, output_path):
    """
    Create a black and white image from hull coordinates in a pandas DataFrame.
    
    Parameters:
    hull_df (pd.DataFrame): DataFrame containing 'x_data' and 'y_data' columns with coordinates
    physical_cell_size (float): Physical size each pixel represents in meters
    output_path (str): Path where the image will be saved
    """
    # Extract coordinates from the DataFrame
    x_coords = hull_df['x_data'].values
    y_coords = hull_df['y_data'].values
    
    # Find the bounds of the data and shift to start at origin
    x_min, x_max = x_coords.min(), x_coords.max()
    y_min, y_max = y_coords.min(), y_coords.max()
    
    # Shift coordinates so minimum values start at 0
    x_shifted = x_coords - x_min
    y_shifted = y_coords - y_min
    
    # Calculate image dimensions in pixels
    width_pixels = int(np.ceil((x_max - x_min) / physical_cell_size))
    height_pixels = int(np.ceil((y_max - y_min) / physical_cell_size))
    
    # Create a black image (0 = black)
    image = np.zeros((height_pixels, width_pixels))
    
    # Create a path from the hull coordinates (shifted to start at origin)
    hull_coords = list(zip(x_shifted / physical_cell_size, y_shifted / physical_cell_size))
    hull_path = Path(hull_coords)
    
    # Create a grid of pixel coordinates
    x_grid, y_grid = np.meshgrid(np.arange(width_pixels), np.arange(height_pixels))
    points = np.column_stack((x_grid.ravel(), y_grid.ravel()))
    
    # Check which points are inside the hull
    inside_hull = hull_path.contains_points(points)
    inside_hull = inside_hull.reshape(height_pixels, width_pixels)
    
    # Set inside points to white (1 = white)
    image[inside_hull] = 1
    
    # Create figure without axes, legends, or any decorations
    fig, ax = plt.subplots(figsize=(width_pixels/100, height_pixels/100), dpi=100)
    ax.imshow(image, cmap='gray', origin='lower', extent=[0, width_pixels, 0, height_pixels])
    
    # Remove all axes, ticks, labels, and whitespace
    ax.set_xlim(0, width_pixels)
    ax.set_ylim(0, height_pixels)
    ax.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    
    # Save the image
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0, dpi=100, facecolor='black')
    plt.close()
    
    print(f"Hull image saved to: {output_path}")
    print(f"Image dimensions: {width_pixels} x {height_pixels} pixels")
    print(f"Physical area covered: {(x_max-x_min):.2f} x {(y_max-y_min):.2f} meters")


In [8]:
xml_folder = "./xml_files"  # Directory containing XML files with tachymeter data
output_folder = "./output_files"  # Directory to save output files
physical_cell_size = 0.01 # Physical size of the cell in meters. Each pixel represents this size.

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
print(f"Output directory ready: {output_folder}")

Output directory ready: ./output_files


In [12]:
for xml_file in os.listdir(xml_folder):
    if xml_file.endswith('.xml'):
        # Construct full path to the XML file
        xml_path = os.path.join(xml_folder, xml_file)
        
        # Process the XML file to extract tachymeter data
        df = load_xml(xml_path)
        
        # Define the output file path based on the XML file name
        base_name = os.path.splitext(xml_file)[0]
        output_path = os.path.join(output_folder, f"{base_name}_hull.png")
        
        # Create the hull image
        save_hull_image(df, physical_cell_size, output_path)
        
        print(f"Processed: {xml_file} -> {base_name}_hull.png")


Hull image saved to: ./output_files\2024_kress_ti_hull.png
Image dimensions: 1352 x 2118 pixels
Physical area covered: 13.51 x 21.17 meters
Processed: 2024_kress_ti.xml -> 2024_kress_ti_hull.png
